In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
import torch.nn as nn
import wandb
import torch
random_seed = 42

In [24]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
#train = pd.read_csv('json_train.csv')
#train = train.drop("Unnamed: 0", axis = 1)
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)


In [25]:
X_train

,r1_x,r1_y,r2_x,r2_y,r3_x,r3_y,r4_x,r4_y,r5_x,r5_y
492,116.0,160.0,72.0,76.0,82.0,166.0,84.0,130.0,90.0,126.0
10529,84.0,168.0,170.0,82.0,130.0,130.0,90.0,164.0,86.0,166.0
366,114.0,110.0,98.0,116.0,72.0,78.0,92.0,118.0,104.0,122.0
3845,92.0,134.0,106.0,86.0,92.0,138.0,92.0,128.0,110.0,78.0
18722,178.0,102.0,168.0,96.0,84.0,134.0,74.0,90.0,102.0,114.0
...,...,...,...,...,...,...,...,...,...,...
22255,74.0,78.0,86.0,82.0,80.0,82.0,78.0,78.0,78.0,86.0
5549,102.0,158.0,104.0,160.0,146.0,78.0,150.0,154.0,102.0,152.0
876,176.0,166.0,168.0,160.0,164.0,156.0,174.0,166.0,72.0,100.0
16284,92.0,138.0,78.0,130.0,122.0,94.0,78.0,130.0,116.0,118.0


In [26]:
train_dataset = ut.MyTensorDataset(torch.FloatTensor(X_train.values), torch.FloatTensor(Y_train.values))
test_dataset = ut.MyTensorDataset(torch.FloatTensor(X_test.values), torch.FloatTensor(Y_test.values))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False,)

In [27]:
size = len(X_train.columns)
net = nn.Sequential(nn.Linear(size, size), nn.LeakyReLU() , nn.Linear(size, 1), nn.Sigmoid())
n_epoch = 20
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)

In [28]:
size

10

In [29]:
loss_history = []

for i in range(n_epoch):
    for x_batch, y_batch in train_dataloader:
        #print(x_batch)
        #y_batch = torch.transpose(y_batch, 0, 1)[0]
        #y_batch = y_batch[0]
        #print(y_batch)
        # Forward
        predictions = net(x_batch)
        #print(predictions)
        loss = criterion(predictions, y_batch)

        # Backward
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

        loss_history.append(loss.item())
    print(f'{i+1}/{n_epoch} current loss: {loss}')

1/20 current loss: 0.6806763410568237
2/20 current loss: 0.6676464080810547
3/20 current loss: 0.6986006498336792
4/20 current loss: 0.7355412840843201
5/20 current loss: 0.7215596437454224
6/20 current loss: 0.7060378789901733
7/20 current loss: 0.6898570656776428
8/20 current loss: 0.6851940751075745
9/20 current loss: 0.6882490515708923
10/20 current loss: 0.689663290977478
11/20 current loss: 0.7030936479568481
12/20 current loss: 0.6996074914932251
13/20 current loss: 0.6900337934494019
14/20 current loss: 0.6724831461906433
15/20 current loss: 0.6895832419395447
16/20 current loss: 0.6897565126419067
17/20 current loss: 0.7010475397109985
18/20 current loss: 0.7140077948570251
19/20 current loss: 0.7033304572105408
20/20 current loss: 0.6861920952796936


In [30]:
test_preds = np.array([])

for x_batch, y_batch in test_dataloader:
    predictions = net(x_batch)
    predictions = torch.transpose(predictions, 0, 1)[0]
    predictions = predictions.detach().numpy()
    test_preds = np.append(test_preds, predictions)

In [31]:
test_preds 

array([0.53087407, 0.53087407, 0.53087407, ..., 0.53087407, 0.53087407,
       0.50463825])

In [32]:
from sklearn.metrics import roc_auc_score
#print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, test_preds)}")

ROC-AUC score : 0.5166310778533824


In [33]:
ut.writeNN(net)